In [3]:
import pandas as pd
df = pd.read_csv("./signed.csv", parse_dates=True)
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu80

Unable to create process using 'C:\Users\Egor Kolyuzhnov\anaconda3-1\python.exe "C:\Users\Egor Kolyuzhnov\anaconda3-1\Scripts\pip-script.py" install torch_nightly -f https://download.pytorch.org/whl/nightly/cu80'


In [2]:
!pip install gensim

In [3]:
from gensim.models import Word2Vec
import time

sentences = []
for i in range(df.shape[0]):
    sentences.append(df['Text'].iloc[i].split(' '))
start = time.time()
model = Word2Vec(sentences=sentences, vector_size=50, window=5, min_count=1, workers=20)
model.save("word2vec.model")
end = time.time()
print(f"{end - start}s")

52.57750844955444s


In [4]:
print(model.wv.most_similar('газ'))

[('электроэнерг', 0.8012456893920898), ('энергоресурс', 0.8004611134529114), ('топлив', 0.7969412803649902), ('нефт', 0.7968339920043945), ('энергоносител', 0.7728245854377747), ('нефтепродукт', 0.7423908710479736), ('пхг', 0.7381792068481445), ('спг', 0.7378959059715271), ('газа»', 0.7263245582580566), ('газов', 0.7114323377609253)]


In [5]:
vectors = []
for text in df['Text']:
    vectors.append([model.wv[w] for w in text.split(' ')])
print(vectors[:1])

[[array([-0.11859299, -0.15629469, -0.15205997, -0.05362122, -0.09970079,
       -0.07330936,  0.01618147,  0.18217646, -0.02097399,  0.07019382,
       -0.01459412, -0.01467434,  0.00171472,  0.00581499, -0.04179226,
        0.00955514,  0.02651698,  0.15097678, -0.05986246, -0.076684  ,
       -0.13371448, -0.08537266,  0.10099266,  0.01670219,  0.00557606,
        0.01318419,  0.07231525, -0.03195928, -0.04666207, -0.09447031,
       -0.13516726, -0.04031296,  0.07559828,  0.02509861,  0.01824079,
       -0.10837547, -0.06542786, -0.06154928, -0.04456985, -0.04861327,
        0.01861917,  0.02400017,  0.08672596, -0.01850136,  0.05548733,
        0.06351005,  0.17679472, -0.02059646,  0.00028067,  0.01158038],
      dtype=float32), array([-3.0472012 ,  0.04800753, -1.373387  ,  1.1955787 , -0.8787327 ,
        0.6225009 , -1.4192338 ,  1.8571931 , -0.5949953 ,  0.9216847 ,
       -0.94021064, -0.43893787, -0.98909974,  3.8201706 , -1.0939885 ,
       -1.9924761 ,  0.90181035,  0.932

In [6]:
!pip install torch

import torch
import torch.nn as nn
import torch.optim as optim

count = int(df.shape[0] * 0.8)
X = vectors
classes = [-1, 0, 1]
y = torch.zeros(len(vectors), 3)
for i in range(len(vectors)):
    y[i][classes.index(df['ChangeSign'].iloc[i])] = 1

In [7]:
trainX, testX = X[:count], X[count:]
trainY, testY = y[:count], y[count:]

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
torch.cuda.is_available()

False

In [ ]:
lstm = nn.LSTM(input_size=50, hidden_size=3, num_layers=1)
optimizer = optim.Adam(lstm.parameters())
loss_fn = nn.MSELoss()

hidden = (
    torch.rand(1, 3),
    torch.rand(1, 3)
)

for epoch in range(1000):
    start = time.time()
    lstm.train()
    y_pred = torch.Tensor([])
    for vs in trainX:
        for v in vs:
            out, hidden = lstm(torch.Tensor(v.reshape(1, -1)).cuda(), hidden)
        y_pred = torch.cat((y_pred, out), 0) 
    loss = loss_fn(y_pred, y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    end = time.time()
    print(f"Epoch {epoch}: Time: {end - start}s")